# Geometrical Methods in Machine Learning

## Homework 2

In [30]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np

from sklearn.datasets import load_digits

### Task 1: Intrinsic dimension estimation (4 points)

Implement $2$ intrinsic dimension estimation methods from a list below. Specify the methods you have chosen in the canvas Homework 2 comments.

Each method should be realized as a Python class implementing public method `fit` as well as providing access to public class variable(s) `dimension` (all methods), `local_dimension`, `dimension_multiscale`, `local_dimension_multiscale` (specific methods).

- `fit` method should return the estimate $\hat{d}$ of the intrinsic dimensionality of the data as well as setting the scalar `dimension` property of the class,
- if the method is _local_, the vector of dimension $n$ `local_dimension` property should be present with `dimension` being their median or mean aggregates along points,
- if the method is _multiscale_, the vector of dimenson $s$ `dimension_multiscale` and the matrix of dimensions $s \times n$ `local_dimension_multiscale` should be present as well, with `dimension` and `local_dimension` being their median or mean aggregates along scales.

Be sure to give a method summary, describing its (see the examples below)
- idea,
- the estimator formula and/or algorithm,
- the reference to pages with estimator derivation, and
- the list of reference(s).

**Grading:** Implementing any single method is awarded with $3$ points, implementing both with $4$ points.

### Intrinsic dimension estimation methods

- [Levina2005] Levina, E., Bickel, P. Maximum Likelihood Estimation of Intrinsic Dimension. _Advances in Neural Information Processing Systems (2005)_

### Examples

In [31]:
# load data
X, y = load_digits(return_X_y=True)

#### MLE

**Idea**

Local intrinsic dimension estimator based on the principle of maximum likelihood to the distances between close neighbors.

**Estimator formula/algorithm**

Let we use $k$ nearest neighbors $\mathbf{x}_1, \dots, \mathbf{k}_n$ of a point $\mathbf{x}$ and $r_j = d(\mathbf{x}, \mathbf{x}_j)$ is the distance from a point $\mathbf{x}$ to the $k$-th nearest neighbor, then the local MLE estimate of the intrinsic dimension is [Levina2005, Eq. 8]

$$\hat{d}_k(\mathbf{x}) = \left[ \frac{1}{k - 1} \sum_{j=1}^{k - 1} \log \frac{r_k(\mathbf{x})}{r_j(\mathbf{x})} \right]^{-1}$$

Averaging over $\hat{d}_k(\mathbf{x})$ gives the intrinsic dimension of the dataset $\mathbf{X}$.

**Estimator derivation**

For derivation see [Levina2005, pp. 3-4].

**References**  
- [Levina2005] Levina, E., Bickel, P. Maximum Likelihood Estimation of Intrinsic Dimension. _Advances in Neural Information Processing Systems (2005)_

In [32]:
from sklearn.neighbors import NearestNeighbors

class MLE():
    
    def __init__(self, k=5, aggregate="mean", eps=1e-6):
        
        super().__init__()
        
        self.k = k
        self.eps = eps
        
        if aggregate=="mean":
            self.aggregate = np.mean
        elif aggregate=="median":
            self.aggregate = np.median
        else:
            raise ValueError("Aggregate function should be 'mean' or 'median'")
        
    def fit(self, X):
        
        # get distances within up to the k-th nearest neighor for each point
        nn = NearestNeighbors(n_neighbors=self.k).fit(X)
        distances, _ = nn.kneighbors()
        distances += self.eps # fix duplicate points, causing d(x_i, x_j)=0
        
        # get local and global intrinsic dimensions
        log_distance_ratios = np.log(distances[:,self.k-1:self.k] / distances[:,:self.k-1])
        self.local_dimension = ((self.k - 2) ** -1 * np.sum(log_distance_ratios, axis=1)) ** -1
        self.dimension = self.aggregate(self.local_dimension)
        
        return self.dimension

In [33]:
# create estimator
estimator = MLE()

# fit estimator, returns (global) intrinsic dimension
dimension = estimator.fit(X)

# get estimates of global and local intrinsic dimensions
dimension = estimator.dimension
local_dimension = estimator.local_dimension

dimension, local_dimension.shape

(8.339146894870613, (1797,))

#### PCA-based

**Idea**

Global dimension estimator corresponding to specified level of explained variance given by the singular value decomposition of the centered data matrix.

**References**  
- [Pearson1901] Pearson, K. On Lines and Planes of Closest Fit to Systems of Points in Space. _Philosophical Magazine (1901)_

In [34]:
from sklearn.decomposition import PCA

class PCAd():
    
    def __init__(self, explained_variance=0.95):
        super().__init__()
        self.explained_variance = explained_variance
        
    def fit(self, X):
        pca = PCA(n_components=self.explained_variance)
        pca.fit(X)
        self.dimension = pca.n_components_

        return self.dimension

In [35]:
# create estimator
estimator = PCAd()

# fit estimator, returns (global) intrinsic dimension
dimension = estimator.fit(X)

# get estimate of (global) dimension
dimension = estimator.dimension
dimension

29

In [ ]:
# your code here

### Task 2: Manifold learning (4 points)

Obtain `Extended Yale B` face dataset ([download](https://github.com/oleg-kachan/GMML2023/blob/main/hw2/data/CroppedYale.zip)) which is comprised of $100 \times 100$ pixels images of $38$ persons times $64$ illumination conditions. Resize images to $32 \times 32$ pixels. You can do it using `Pillow` ([link](https://pillow.readthedocs.io/), tested) or any other image processing library of your choice.

1. Estimate the intrinsic dimensionality with a method(s) of your choice (methods from task 1 are perfect candidates) and perform dimensionality reduction to the intrinsic dimension $\hat{d}$ and dimensions $2$ or $3$ for visualization purposes using manifold learning methods of your choice.

2. Compute NPR (neigborhood preservation ratio, see [seminar 4](https://github.com/oleg-kachan/GMML2022/blob/main/seminar4/seminar4_solution.ipynb)) of algorithms you have used for 2 different values of $d = \{2$ or $3, \hat{d} \}$ and fixed number of nearest neighbors $k$. 

3. Explore the embedding space of dimension $2$ or $3$ for clusters and meaningful interpretations, comment the possible meaning of the new coordinates.

4. Compute and plot persistence diagrams for dimensions $0$ and $1$, conclude whether the dataset have untrivial topology such a several clusters and/or presence of cycles in dimension $1$. If applicable, how can one explain a presence of cycles for the `Extended Yale B` face dataset?

**Grading:** Each subtask is awarded with $1$ point.

In [ ]:
# your code here

#### Grading:

- 8/10 points are awarded for completing all the tasks and giving proper answers to questions.
- 2/10 points are awarded for the quality of reporting, be sure to give explanations and comments to your solutions.
- +1 extra point may be awarded for the extra work performed, be creative.